
***Zadanie 1***: *Drzewo genealogiczne*

+ działające w Swish'u

In [1]:
% Fakty
ojciec(jan, anna).
ojciec(jan, piotr).
matka(maria, anna).
matka(maria, piotr).
malzenstwo(jan, maria).
mezczyzna(jan).
kobieta(maria).
mezczyzna(piotr).
kobieta(anna).

% Reguła: przodek(X, Y)
przodek(X, Y) :-
    ojciec(X, Y).                    % Ojciec jest przodkiem
przodek(X, Y) :-
    matka(X, Y).                     % Matka jest przodkiem
przodek(X, Y) :-
    ojciec(X, Z), przodek(Z, Y).     % Jeśli X jest ojcem Z, a Z jest przodkiem Y
przodek(X, Y) :-
    matka(X, Z), przodek(Z, Y).      % Jeśli X jest matką Z, a Z jest przodkiem Y

% Reguła: brat(X, Y)
brat(X, Y) :-
    ojciec(Z, X), ojciec(Z, Y),      % X i Y mają tego samego ojca
    matka(W, X), matka(W, Y),        % X i Y mają tę samą matkę
    X \= Y,                          % X i Y to różne osoby
    mezczyzna(X).                    % X jest mężczyzną

% Reguła: siostra(X, Y)
siostra(X, Y) :-
    ojciec(Z, X), ojciec(Z, Y),      % X i Y mają tego samego ojca
    matka(W, X), matka(W, Y),        % X i Y mają tę samą matkę
    X \= Y,                          % X i Y to różne osoby
    kobieta(X).                      % X jest kobietą

% Reguła: kuzyn(X, Y)
kuzyn(X, Y) :-
    ojciec(Z, X), ojciec(W, Y),      % Ojciec X i ojciec Y są braćmi
    brat(Z, W),
    X \= Y.                          % X i Y to różne osoby
kuzyn(X, Y) :-
    matka(Z, X), matka(W, Y),        % Matka X i matka Y są siostrami
    siostra(Z, W),
    X \= Y.   		% X i Y to różne osoby

% Przykładowe zapytania i oczekiwane wyniki:
    ?- przodek(jan, anna). % Oczekiwane: true
    ?- brat(piotr, anna). % Oczekiwane: true
    ?- kuzyn(X, piotr). % Oczekiwane: X = Y, gdzie Y to kuzyni Piotra


***Zadanie 2***: *System rezerwacji lotów*

+ działające w Swi-prolog

In [ ]:
% Fakty
lotnisko(warszawa).
lotnisko(krakow).
lotnisko(gdansk).
polaczenie(warszawa, krakow, 300).
polaczenie(krakow, gdansk, 200).
polaczenie(warszawa, gdansk, 500).

% Reguły
polaczenie(X, Y) :-
    polaczenie(X, Y, _).  % Bezpośrednie połączenie w jednym kierunku
polaczenie(X, Y) :-
    polaczenie(Y, X, _).  % Bezpośrednie połączenie w przeciwnym kierunku

% Bezpośrednie połączenie
koszt_podrozy(X, Y, C) :-
    polaczenie(X, Y, C).

% Połączenie pośrednie
koszt_podrozy(X, Y, C) :-
    polaczenie(X, Z, C1),     % Połączenie z X do Z
    Z \= Y,                   % Miasto pośrednie Z musi być różne od Y
    koszt_podrozy(Z, Y, C2),  % Rekurencyjnie szukamy kosztu od Z do Y
    C is C1 + C2.             % Sumujemy koszty


% Najkrótsza trasa
najkrotsza_droga(X, Y, Sciezka, C) :-
    setof([Koszt, Trasa], trasa(X, Y, Trasa, Koszt), Wyniki),
    Wyniki = [[C, Sciezka] | _].

% Przykładowe zapytania i oczekiwane wyniki:
    ?- polaczenie(warszawa, krakow). % Oczekiwane: true

    ?- koszt_podrozy(warszawa, gdansk, C). % Oczekiwane: C = 500

    ?- najkrotsza_droga(warszawa, gdansk). % Oczekiwane: trasa przez Kraków, koszt 500

***Zadanie 3***: *System zarządzania magazynem*

+ działające w Swish'u

In [3]:
% Fakty
produkt(tv, 10, sektor_a).
produkt(pralka, 5, sektor_b).
produkt(lodowka, 3, sektor_c).

% Reguła przenoszenia produktu 1
dostepny_produkt(X) :-
    produkt(X, Ilosc, _),
    Ilosc > 0.

% Reguła przenoszenia produktu 2
:- dynamic produkt/3.
przenies_produkt(X, Y) :-
    retract(produkt(X, Ilosc, _)),
    assertz(produkt(X, Ilosc, Y)).

% Reguła uzupełniania stanu
uzupelnij_stan(X, N) :-
    retract(produkt(X, Ilosc, Sektor)),
    NowaIlosc is Ilosc + N,
    assertz(produkt(X, NowaIlosc, Sektor)).

% Przykładowe zapytania i oczekiwane wyniki:

    ?- dostepny_produkt(tv). : true

    ?- przenies_produkt(tv, sektor_b). : produkt(tv, 10, sektor_b)

    ?- uzupelnij_stan(pralka, 3). : produkt(pralka, 8, sektor_b)


***Zadanie 4***: *System planowania trasy kuriera*

+ działające w Swi-prolog

In [ ]:
% Fakty
miasto(warszawa).
miasto(krakow).
miasto(poznan).
droga(warszawa, krakow, 4).
droga(krakow, poznan, 3).
droga(warszawa, poznan, 7).



% Reguła drogi
droga(X, Y) :-
    droga(X, Y, _).    % Bezpośrednie połączenie w jednym kierunku
droga(X, Y) :-
    droga(Y, X, _).    % Bezpośrednie połączenie w przeciwnym kierunku

% Bezpośrednie połączenie
czas_przejazdu(X, Y, T) :-
    droga(X, Y, T).

czas_przejazdu(X, Y, T) :-
    droga(X, Z, T1),           % Istnieje droga z X do Z
    Z \= Y,                    % Z nie może być tym samym miastem co Y
    czas_przejazdu(Z, Y, T2),  % Rekurencyjnie obliczamy czas od Z do Y
    T is T1 + T2.              % Sumujemy czas przejazdu

% Reguła odwiedzonych lotnisk
droga(X, Y, [X, Y], C) :-
   polaczenie(X, Y, C).

droga(X, Y, [X|Trasa], C) :-
   polaczenie(X, Z, C1),
   \+ member(Z, [X|Trasa]),  % Sprawdzamy, czy Z nie jest już w bieżącej trasie
   droga(Z, Y, Trasa, C2),
   C is C1 + C2.

% Pomocnicza reguła: Znalezienie trasy i czasu
trasa(X, Y, [X, Y], T) :-
    droga(X, Y, T).           % Bezpośrednia droga

trasa(X, Y, [X | Sciezka], T) :-
    droga(X, Z, T1),          % Istnieje droga z X do Z
    Z \= Y,                   % Z nie może być tym samym miastem co Y
    \+ member(Z, [X]),        % Unikamy pętli (miasto Z nie może być już odwiedzone)
    trasa(Z, Y, Sciezka, T2), % Rekurencyjnie szukamy trasy od Z do Y
    T is T1 + T2.             % Sumujemy czas przejazdu

% Główna reguła: Znalezienie najkrótszej trasy
najkrotsza_trasa(X, Y, Sciezka, T) :-
    setof([Czas, Trasa], trasa(X, Y, Trasa, Czas), Wyniki),
    Wyniki = [[T, Sciezka] | _].  % Wybieramy trasę z najmniejszym czasem

% Przykładowe zapytania i oczekiwane wyniki:
    ?- polaczenie(warszawa, krakow). % Oczekiwane: true

    ?- czas_przejazdu(warszawa, poznan, T). % Oczekiwane: T = 7

    ?- najkrotsza_trasa(warszawa, gdansk). % Oczekiwane: trasa przez Kraków, koszt 500

***Zadanie 5***: *System rekomendacji książek*

+ działające w Swish'u

In [ ]:
% Fakty
ksiazka('W pustyni i w puszczy', sienkiewicz, przygodowa).
ksiazka('Hobbit', tolkien, fantasy).
ksiazka('Pan Tadeusz', mickiewicz, poezja).
ksiazka('Zibi', kolton, biografia).
ocena(uzytkownik1, 'W pustyni i w puszczy', 5).
ocena(uzytkownik1, 'Hobbit', 4).
ocena(uzytkownik2, 'Pan Tadeusz', 3).
ocena(uzytkownik2, 'Zibi', 5).
preferencje(uzytkownik1, fantasy).
preferencje(uzytkownik1, przygodowa).
preferencje(uzytkownik2, biografia).

% Reguły
:- dynamic ocena/3.

polecane_ksiazki(X, Gatunek) :-
    preferencje(X, Gatunek),
    findall(Ksiazka,(ksiazka(Ksiazka, _, Gatunek),
    ocena(X, Ksiazka, Ocena),Ocena >= 4), Książki),
    format('Polecane książki dla ~w w gatunku ~w: ~w~n', [X, Gatunek, Książki]).

dodaj_ocene(Uzytkownik, Ksiazka, Ocena) :-
    assertz(ocena(Uzytkownik, Ksiazka, Ocena)),
    format('Ocena dla książki "~w" została dodana przez ~w. Ocena: ~w~n', [Ksiazka, Uzytkownik, Ocena]).

usun_ocene(Uzytkownik, Ksiazka) :-
    retract(ocena(Uzytkownik, Ksiazka, _)),
    format('Ocena dla książki "~w" została usunięta przez ~w.~n', [Ksiazka, Uzytkownik]).


% Przykładowe zapytania i oczekiwane wyniki:

    ?- polecane_ksiazki(uzytkownik1, fantasy). : ['Hobbit']

    ?- polecane_ksiazki(uzytkownik2, biografia). : ["Zibi"]